In [1]:
import pandas as pd
import numpy as np
import sys  
sys.path.insert(0, '../source/')
from preprocess_data_utils import *

In [2]:
#load real light curves
snIas=None
for i in range(11):
    snIas_str = '../data/testing/snIa_{}.csv'.format(i)
    print(snIas_str)
    snIa = pd.read_csv(snIas_str,sep="|").dropna(axis=1)
    if snIas is None:
        snIas = pd.concat([snIas, snIa],ignore_index=True)
    else:
        snIas = snIa

../data/testing/snIa_0.csv
../data/testing/snIa_1.csv
../data/testing/snIa_2.csv
../data/testing/snIa_3.csv
../data/testing/snIa_4.csv
../data/testing/snIa_5.csv
../data/testing/snIa_6.csv
../data/testing/snIa_7.csv
../data/testing/snIa_8.csv
../data/testing/snIa_9.csv
../data/testing/snIa_10.csv


In [3]:
snIas

,Unnamed: 0,objectId,jd,magpsf,sigmapsf
NaN,ZTF18aawmxdx,2.458462e+06,18.7827,0.101659,1
NaN,ZTF18aawmxdx,2.458465e+06,17.9839,0.065190,1
NaN,ZTF18aawmxdx,2.458465e+06,18.0191,0.058467,2
NaN,ZTF18aawmxdx,2.458469e+06,17.6659,0.058305,2
NaN,ZTF18aawmxdx,2.458472e+06,17.5561,0.056013,2
...,...,...,...,...,...
NaN,ZTF18adbhrjs,2.458495e+06,18.7096,0.080200,1
NaN,ZTF18adbhrjs,2.458503e+06,19.0431,0.106514,2
NaN,ZTF18adbhrjs,2.458509e+06,19.3554,0.209057,2
NaN,ZTF18adbhrjs,2.458512e+06,19.4559,0.118193,2


In [4]:
#rename columns
snIas.columns = ["id","time","flux","flux_err","band"]

In [5]:
snIas

,id,time,flux,flux_err,band
NaN,ZTF18aawmxdx,2.458462e+06,18.7827,0.101659,1
NaN,ZTF18aawmxdx,2.458465e+06,17.9839,0.065190,1
NaN,ZTF18aawmxdx,2.458465e+06,18.0191,0.058467,2
NaN,ZTF18aawmxdx,2.458469e+06,17.6659,0.058305,2
NaN,ZTF18aawmxdx,2.458472e+06,17.5561,0.056013,2
...,...,...,...,...,...
NaN,ZTF18adbhrjs,2.458495e+06,18.7096,0.080200,1
NaN,ZTF18adbhrjs,2.458503e+06,19.0431,0.106514,2
NaN,ZTF18adbhrjs,2.458509e+06,19.3554,0.209057,2
NaN,ZTF18adbhrjs,2.458512e+06,19.4559,0.118193,2


In [6]:
def check_lc_length(data):
    #different ids for different passbands
    data_cp = data.copy()
    data_cp['ob_p']=data.id+data.band.astype('str')
    #get dataframe with min and max mjd values per each object id
    group_by_mjd = data_cp.groupby(['id'])['time'].agg(['min', 'max']).rename(columns = lambda x : 'time_' + x).reset_index()
    group_by_mjd["time_diff"]=group_by_mjd.time_max-group_by_mjd.time_min
    return group_by_mjd

In [7]:
t_d=check_lc_length(snIas)


In [8]:
t_d.time_diff.describe()
#ufff muy muy poquitas obs

count    100.000000
mean      76.564685
std       89.617220
min        5.980046
25%       33.972297
50%       46.021863
75%       75.810431
max      465.796516
Name: time_diff, dtype: float64

In [9]:
snIa_tags = np.zeros(snIas.shape[0],dtype='int32')
snIas["tag"]=snIa_tags
snIas

,id,time,flux,flux_err,band,tag
NaN,ZTF18aawmxdx,2.458462e+06,18.7827,0.101659,1,0
NaN,ZTF18aawmxdx,2.458465e+06,17.9839,0.065190,1,0
NaN,ZTF18aawmxdx,2.458465e+06,18.0191,0.058467,2,0
NaN,ZTF18aawmxdx,2.458469e+06,17.6659,0.058305,2,0
NaN,ZTF18aawmxdx,2.458472e+06,17.5561,0.056013,2,0
...,...,...,...,...,...,...
NaN,ZTF18adbhrjs,2.458495e+06,18.7096,0.080200,1,0
NaN,ZTF18adbhrjs,2.458503e+06,19.0431,0.106514,2,0
NaN,ZTF18adbhrjs,2.458509e+06,19.3554,0.209057,2,0
NaN,ZTF18adbhrjs,2.458512e+06,19.4559,0.118193,2,0


In [10]:
enough_td = t_d.id[t_d.time_diff>=30].values
snIas=snIas[snIas.id.isin(enough_td)]
len(snIas.id.unique())

81

In [11]:
snIbcs_str='../data/testing/snIbc.csv'
snIbcs = pd.read_csv(snIbcs_str,sep="|").dropna(axis=1)
#rename columns
snIbcs.columns = ["id","time","flux","flux_err","band"]
snIbcs

,id,time,flux,flux_err,band
0,ZTF18aakpigz,2.458280e+06,19.8569,0.167994,2
1,ZTF18aakpigz,2.458824e+06,17.8664,0.243286,1
2,ZTF18aakpigz,2.458828e+06,18.2403,0.092048,2
3,ZTF18aakpigz,2.458831e+06,18.2449,0.084126,1
4,ZTF18aakpigz,2.458834e+06,18.1861,0.106567,2
...,...,...,...,...,...
1166,ZTF20aagewqr,2.458865e+06,19.0348,0.090412,2
1167,ZTF20aagewqr,2.458875e+06,18.5037,0.086205,2
1168,ZTF20aagewqr,2.458875e+06,19.7588,0.164363,1
1169,ZTF20aagewqr,2.458876e+06,18.6594,0.089834,2


In [12]:
t_d=check_lc_length(snIbcs)
t_d.time_diff.describe()

count     51.000000
mean      88.544626
std      123.870959
min        1.055995
25%       27.852147
50%       45.981597
75%       96.800081
max      603.143588
Name: time_diff, dtype: float64

In [13]:
snIbc_tags = np.ones(snIbcs.shape[0],dtype='int32')
snIbcs["tag"]=snIbc_tags
snIbcs

,id,time,flux,flux_err,band,tag
0,ZTF18aakpigz,2.458280e+06,19.8569,0.167994,2,1
1,ZTF18aakpigz,2.458824e+06,17.8664,0.243286,1,1
2,ZTF18aakpigz,2.458828e+06,18.2403,0.092048,2,1
3,ZTF18aakpigz,2.458831e+06,18.2449,0.084126,1,1
4,ZTF18aakpigz,2.458834e+06,18.1861,0.106567,2,1
...,...,...,...,...,...,...
1166,ZTF20aagewqr,2.458865e+06,19.0348,0.090412,2,1
1167,ZTF20aagewqr,2.458875e+06,18.5037,0.086205,2,1
1168,ZTF20aagewqr,2.458875e+06,19.7588,0.164363,1,1
1169,ZTF20aagewqr,2.458876e+06,18.6594,0.089834,2,1


In [14]:
enough_td = t_d.id[t_d.time_diff>30].values
snIbcs=snIbcs[snIbcs.id.isin(enough_td)]
len(snIbcs.id.unique())

37

In [15]:
snIIn_str='../data/testing/snIIn.csv'
snIIn = pd.read_csv(snIIn_str,sep="|").dropna(axis=1)
#rename columns
snIIn.columns = ["id","time","flux","flux_err","band"]
snIIn

,id,time,flux,flux_err,band
0,ZTF18aakrnvd,2.458277e+06,18.5568,0.039183,2
1,ZTF18aakrnvd,2.458280e+06,18.5092,0.039007,2
2,ZTF18aakrnvd,2.458283e+06,18.5405,0.041797,2
3,ZTF18aakrnvd,2.458289e+06,18.5419,0.075994,2
4,ZTF18aakrnvd,2.458292e+06,18.5524,0.090697,2
...,...,...,...,...,...
1880,ZTF20aadtarr,2.458868e+06,18.2107,0.063649,2
1881,ZTF20aadtarr,2.458868e+06,18.1221,0.078859,1
1882,ZTF20aadtarr,2.458873e+06,18.0380,0.060230,1
1883,ZTF20aadtarr,2.458876e+06,17.9971,0.071077,1


In [19]:
t_d=check_lc_length(snIIn)
enough_td = t_d.id[t_d.time_diff>=30].values
snIIn=snIIn[snIIn.id.isin(enough_td)]
len(snIIn.id.unique())

37

In [17]:

t_d.time_diff.describe()

count     43.000000
mean     168.741152
std      181.236652
min       10.017928
25%       48.901493
50%       93.946169
75%      215.135197
max      608.253113
Name: time_diff, dtype: float64

In [21]:
snIIp_str='../data/testing/snIIP.csv'
snIIp = pd.read_csv(snIIp_str,sep="|").dropna(axis=1)
#rename columns
snIIp.columns = ["id","time","flux","flux_err","band"]

t_d=check_lc_length(snIIp)
enough_td = t_d.id[t_d.time_diff>=30].values
snIIp=snIIp[snIIp.id.isin(enough_td)]
len(snIIp.id.unique())

23

In [22]:
t_d.time_diff.describe()

count     24.000000
mean     146.474773
std      139.674586
min        4.034768
25%       59.669893
50%       94.403530
75%      153.670113
max      497.723299
Name: time_diff, dtype: float64